## Mathematik für Biologiestudierende II

Sommersemester 2024

25.06.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf
import seaborn as sns
sns.set_theme()

# Lineare Modelle

#### Themen heute

* mehrere erklärende Variable
* Transformationen
* Normalverteilungsannahmen
* Varianten von `smf.ols`

# Mehrere Erklärende Variable

In [ ]:
df = pd.read_csv('larven.csv')
df.head()

* AnzLarven:  Anzahl der Larven eines Kleinstlebewesens pro Liter
* A, B, C, D, E:  Konzentrationen eines potentiellen Schadstoffs in ppb (Teile pro Milliarde)

* das Experiment ist *beobachtend*
* keine Möglichkeit der unabhängigen Veränderung einzelner Parameter

In [ ]:
df.describe().round(0)

In [ ]:
formel = 'AnzLarven ~ A + B + C + D + E'
modell = smf.ols(formel, df)

In [ ]:
res = modell.fit()

In [ ]:
res.summary()

* Der Einfluss von B und E ist nicht signifikant
* Wir entfernen sie aus dem Modell

In [ ]:
formel2 = 'AnzLarven ~ A + C + D'
modell2 = smf.ols(formel2, df)
res2 = modell2.fit()

In [ ]:
res2.summary()

* die drei verbleibenden Stoffe haben signifikanten Einfluss auf die Larvenpopulation
* C und D verringen die Anzahl: Schadstoffe
* A erhöht sie:  Nährstoff

# Transformationen

* Das Konfidenzintervall für den Koeffizienten des Stoffs D wir angegeben als [-0., -0.]
* Lösung:  wir geben die Konzentration von D statt in ppb in ppm an
* Das multipliziert den Koeffizienten mit 1000

In [ ]:
df['D_in_ppm'] = df.D / 1000

In [ ]:
formel3 = 'AnzLarven ~ A + C + D_in_ppm'
modell3 = smf.ols(formel3, df)
res3 = modell3.fit()

In [ ]:
res3.summary()

## Transformation

* wir haben eine Spalte der Matric transformiert von ppb auf ppm
* dadurch wurd die Statistik nicht verändert; das Ergebnis wurde nur anschaulicher

* einzelne Zeilen können nicht transformiert werden, alle Zeilen müssen in demselbe System gemessen werden

## Beispiel: Galapagos-Inseln

* Daten aus Faraway:  Linear Models with Python
* ursprüngliche Datenquelle
  * Johnson, M., and Raven, P.:  *Species Number and endemism: the Gálapagos Archipelago revisited.*  Science 179 (1973), 893-895

In [ ]:
df = pd.read_csv('galapagos.csv')
df.head()

* Species:  Anzahl verschiedener Wirbeltierarten
* Area:  Größe der Insel
* Elevation:  Höchste Erhebung auf der Insel
* Nearest:  Abstand zur nächsten Insel
* Scruz:  Abstand zu Santa Cruz
* Adjacent:  Größe der nächstgelegenen Insel

In [ ]:
df.describe()

In [ ]:
formel = 'Species ~ Area + Elevation + Nearest + Scruz + Adjacent'
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

Nur zwei der erklärenden Variablen haben signifikanten Einfluss

* Die Höhe der Insel
* Die Fläche der benachbarten Insel, aber mit negativer Korrelation

# Normalverteilungsannahmen

* `smf.ols` hat Anwendungsvoraussetzungen
* eine davon ist, dass alle Variablen normalverteilt sind

* wir prüfen das mit qq-Plots wie in Lektion 

In [ ]:
import statsmodels.api as sm
pp_s = sm.ProbPlot(df.Species)
pp_s.qqplot();

Faraway schlägt vor, die Wurzel aus Species zu verwenden

In [ ]:
pp_sw = sm.ProbPlot(np.sqrt(df.Species))
pp_sw.qqplot();

Auch der Logarithmus wäre eine einleuchtende Wahl

In [ ]:
pp_lw = sm.ProbPlot(np.log(df.Species))
pp_lw.qqplot();

Ich zeige alle QQ-Plots in einem Bild mit einem Verfahren, welches nicht zum Stoff gehört

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()
for j in range(6):
    spalte = df.columns[j+1]
    ax = fig.add_subplot(231+j)
    pp = sm.ProbPlot(df[spalte])
    pp.qqplot(ax=ax, xlabel=spalte, ylabel='')
fig.subplots_adjust(wspace=0.5, hspace=0.4)

Also verletzen alle Variablen die Normalverteilungsannahme mehr oder weniger deutlich


In [ ]:
log_formel = 'np.log(Species) ~ np.log(Area) + np.log(Elevation) + np.log(Nearest) + np.log(Scruz) + np.log(Adjacent)'
log_modell = smf.ols(log_formel, df)

* Santa Cruz hat Abstand 0 zu Santa Cruz
* Wir nehmen die Wurzel aus dieser Variablen

In [ ]:
log_formel = 'np.log(Species) ~ np.log(Area) + np.log(Elevation) + np.log(Nearest) + np.sqrt(Scruz) + np.log(Adjacent)'
log_modell = smf.ols(log_formel, df)

In [ ]:
log_res = log_modell.fit()
log_res.summary()

Nur noch die Fläche hat einen signifikanten Einfluss

In [ ]:
log_formel2 = 'np.log(Species) ~ np.log(Area)'
log_modell2 = smf.ols(log_formel2, df)
log_res2 = log_modell2.fit()

In [ ]:
log_res2.summary()

In [ ]:
w_formel = 'np.sqrt(Species) ~ np.sqrt(Area) + np.sqrt(Elevation) + np.sqrt(Nearest) + np.sqrt(Scruz) + np.sqrt(Adjacent)'
w_modell = smf.ols(w_formel, df)
w_res = w_modell.fit()

In [ ]:
w_res.summary()

In [ ]:
log_formel = 'np.log(Species) ~ np.log(Area) + np.log(Elevation) + np.sqrt(Scruz) + np.log(Adjacent)'
log_modell = smf.ols(log_formel, df)
res = log_modell.fit()
res.summary()

In [ ]:
log_formel = 'np.log(Species) ~ np.log(Area) + np.log(Elevation) + np.sqrt(Scruz)'
log_modell = smf.ols(log_formel, df)
res = log_modell.fit()
res.summary()

In [ ]:
log_formel = 'np.log(Species) ~ np.log(Area) + np.sqrt(Scruz)'
log_modell = smf.ols(log_formel, df)
res = log_modell.fit()
res.summary()

In [ ]:
df = pd.read_csv('galton.csv')
df.head()

In [ ]:
formel = 'childHeight ~ father + mother + gender'
modell = smf.ols(formel, df)

In [ ]:
res = modell.fit()

In [ ]:
res.summary()